In [66]:
import tensorflow as tf
import tensorflow_datasets as tdfs
import tensorflow_ranking as tfr
import cv2
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.utils import load_img
from tensorflow.keras.models import load_model
from PIL import Image
from tqdm import tqdm
import os
from collections import namedtuple

In [67]:
def load_image(path): #reading the images from the given path as bytes and decoding them to create a tensorflow data pipeline
    byte = tf.io.read_file(path)
    image = tf.io.decode_jpeg(byte)
    return image

In [68]:
test_images = tf.data.Dataset.list_files('augmented_data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [69]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = 'utf-8') as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

In [70]:
test_labels = tf.data.Dataset.list_files('augmented_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [71]:
test = tf.data.Dataset.zip((test_images, test_labels)).shuffle(1500).batch(900)

In [72]:
face_detector = load_model('face_detector')
face_detector.summary()

Model: "face_detector"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        [(None, 1),               16826181  
                              (None, 4)]                         
                                                                 
Total params: 16,826,181
Trainable params: 16,826,181
Non-trainable params: 0
_________________________________________________________________


In [73]:
random_test_images = test.as_numpy_iterator()
res = random_test_images.next()
sample_img = res[0]
sample_coordinates = res[1][1]
y_pred_multiple = face_detector.predict(sample_img.reshape(900, 120, 120, 3))

29/29 [==============================] - 13s 447ms/step


In [74]:
y_pred_multiple[1].shape

(900, 4)

In [75]:
test_ = tf.data.Dataset.zip((test_images, test_labels)).shuffle(1500).batch(1)
random_test_images = test_.as_numpy_iterator()
res = random_test_images.next()
sample_img = res[0]
sample_coordinates = res[1][1]
y_pred = face_detector.predict(sample_img.reshape(1, 120, 120, 3))

1/1 [==============================] - 0s 55ms/step


In [76]:
list(y_pred[1][0]), list(sample_coordinates[0])

([0.35036597, 0.036909565, 0.9975743, 0.98846954], [0.3464, 0.0257, 1.0, 1.0])

In [77]:
map_metric = tfr.keras.metrics.MeanAveragePrecisionMetric(topn=2)
bb_intersection_over_union(list(sample_coordinates[0]), list(y_pred[1][0]))

0.9679786085615781

In [78]:
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

def intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = (xB - xA) * (yB - yA)

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [86]:
test_ = tf.data.Dataset.zip((test_images, test_labels)).shuffle(1500).batch(1)
correct_class = 0
random_test_images = test_.as_numpy_iterator()
incorrect_indexes = []
avg_bbox_acc_sum = 0
for a in tqdm(range(len(test_labels))):
    res = random_test_images.next()
    sample_img = res[0]
    sample_class = res[1][0]
    sample_coordinates = res[1][1]
    y_pred = face_detector.predict(sample_img.reshape(1, 120, 120, 3), verbose=0)
    
    pred_class = 1
    if y_pred[0][0][0] < 0.9:
        pred_class = 0
        
    if sample_class[0][0] == pred_class:
        correct_class += 1
        if pred_class == 1:
            map_metric = tfr.keras.metrics.MeanAveragePrecisionMetric(topn=2)
            avg_bbox_acc_sum += (intersection_over_union(list(sample_coordinates[0]), list(y_pred[1][0])) * 100)
    else:
        incorrect_indexes.append(a)    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [01:08<00:00, 13.20it/s]


In [87]:
avg_bbox_acc_sum 

65853.62474248659

In [88]:
correct_class, len(test_labels)

(896, 900)

In [93]:
# correct face detection
class_pred_accuracy = (correct_class * 100) / len(test_labels)
class_pred_accuracy

99.55555555555556

In [90]:
incorrect_indexes

[191, 309, 715, 877]

In [92]:
# Mean average precision 
avg_bbox_acc_sum / len(test_labels)

73.17069415831843